In [225]:
import numpy as np
import pandas as pd
import itertools as it
import pickle
import glob
import os
import string
import gc
import re
import time
import nltk
import spacy
import textacy
#import en_core_web_md
import sematch
import gensim

from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from tqdm import tqdm, tqdm_notebook
from scipy import sparse
from scipy.optimize import minimize
    
from clean_utils import *
from parser_utils import *
%load_ext autoreload
%autoreload 2

from twit_prepocess import tokenize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [226]:
import regex as re
from spacy.tokenizer import Tokenizer

prefix_re = spacy.util.compile_prefix_regex(tuple(i for i in nlp.Defaults.prefixes if i not in '<'))
suffix_re = spacy.util.compile_suffix_regex(tuple(i for i in nlp.Defaults.suffixes if i not in '>'))
infix_re = spacy.util.compile_infix_regex(tuple(i for i in nlp.Defaults.infixes if i not in '<'))

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer)

nlp = spacy.load('en')
nlp.tokenizer = custom_tokenizer(nlp)

In [227]:
def clean_lemmat_twitter(df, shapes):
    t = time.time()
    print('clean and twitter tocken', df.shape)
    df = df.apply(lambda x: clean(x))
    df = df.apply(lambda x :tokenize(x))
    df_train_tw = df.iloc[:shapes[0]]
    df_test_tw = df.iloc[shapes[0]:]
    print('Lemmatizing text.', df.shape)
    q1 = []
    for doc in tqdm(nlp.pipe(df, n_threads=8, batch_size=10000)):
        word_list = []
        for c in doc:
            if c.lemma_ !='-PRON-':
                word_list.append(c.lemma_)
            else:
                word_list.append(c.text)
        q1.append(' '.join(i for i in word_list))
    q1 = pd.DataFrame(q1, columns=['comment_text'])
    print('Cleaning based on forums functions.', df.shape)
    q1.comment_text = q1.comment_text.apply(lambda x: (' '.join([correction(i) for i in x.split(' ')])))
    df_train_correct = q1.iloc[:shapes[0]]
    df_test_correct = q1.iloc[shapes[0]:]
    print('Text cleaning done, time it took:', time.time() - t, q1.shape)
    return df_train_correct, df_test_correct, df_train_tw , df_test_tw

In [228]:
NROWS = None

In [229]:
data =  pd.read_csv('data/train.csv', sep=',', index_col='id', nrows=NROWS)
shapes = data.shape
data['train'] = 1
data = pd.concat([data, pd.read_csv('data/test.csv', sep=',', index_col='id', nrows=NROWS)], axis=0)
data.train.fillna(0, inplace=True)

In [8]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec('vocs/glove.twitter.27B.200d.txt', 'vocs/glove.twitter.27B.200d_w2v.txt')

(1193514, 200)

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('vocs/glove.twitter.27B.200d_w2v.txt')

words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

In [216]:
WORDS["am"]

225

In [5]:
import joblib

In [11]:
joblib.dump(WORDS, 'vocs/words_glove_twitter_for_check_spell.pkl')

['vocs/words_glove_twitter_for_check_spell.pkl']

In [6]:
WORDS = joblib.load('vocs/words_glove_twitter_for_check_spell.pkl')

In [230]:
df_train_corr, df_test_corr, df_train_tw, df_test_tw = clean_lemmat_twitter(data.comment_text, shapes)

clean and twitter tocken (312735,)


0it [00:00, ?it/s]

Lemmatizing text. (312735,)


312735it [1:09:17, 75.21it/s]


Cleaning based on forums functions. (312735,)
Text cleaning done, time it took: 4345.021574258804 (312735, 1)


In [231]:
df_train_corr.shape, df_test_corr.shape, df_train_tw.shape, df_test_tw.shape

((159571, 1), (153164, 1), (159571,), (153164,))

In [224]:
i = 3
print(data.comment_text[i])
print(df_train_corr.comment_text[i])
print(tokenize(data.comment_text[i]))

"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.

There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "
" more i cannot make any real suggestion on improvement - i wonder if the section statistic should be later on , or a subsection of " " type of accident " " - i think the reference may need tidy so that they be all in the exact same format ie date format etc . i can do that later on , if no - one else do first - if you have any preference for format style on reference or wan

In [232]:
df_train_corr.to_csv('pre_data/df_train_tw_corr.csv', index = True, sep=';', index_label='index')
df_test_corr.to_csv('pre_data/df_test_tw_corr.csv', index = True, sep=';', index_label='index')
df_train_tw.to_csv('pre_data/df_train_tw.csv', index = True, sep=';', index_label='index')
df_test_tw.to_csv('pre_data/df_test_tw.csv', index = True, sep=';', index_label='index')

In [30]:
train = pd.read_csv('pre_data/df_train_spacycorr.csv', sep=';')
print(train.shape)
train.head()

(159571, 2)


,index,comment_text
0,0,explanationwhy edit make username hardcore met...
1,1,d'aww match background colour i be seemingly s...
2,2,hey man i be really try edit war -PRON- be guy...
3,3,morei can not make real suggestion improvement...
4,4,sir hero chance remember page that be
